# Generation/Consumption Analysis by Linear Programming

by Jeffrey Kantor  (jeff at nd.edu). The latest version of this notebook is available at [https://github.com/jckantor/CBE20255](https://github.com/jckantor/CBE20255). 

### Summary

Generation/Consumption analysis combines technologically feasible chemical reactions into processes for the economic production of chemical goods. This notebook demonstrates generation/consumption analysis using linear programming techniques in Python. The example uses the `PuLP`, a linear programming toolkit for linear programming.

## Problem Description

### Solar Thermochemical Production of Ammonia

Ammonia has a wide range of commercial uses, including an essential role as a fertilizer for the global agriculture. The principle means of production is the Haber/Bosch process where hydrogen and nitrogren react at high temperature and pressure over an iron catalyst, and the required hydrogen is produced by the steam reforming of methane from natural gas. Globally, this process consumes 5% of all natural gas produced, and 2% of all energy resources.

Recently, several research groups have proposed alternative routes to the production of ammonia using solar radiation as the major source of process energy. Among them, the [two step production of ammonia via Al_2O_3/AlN\\)]() offers the advantage of atmospheric operation production of a synthesis gas by-product. The process consists of the highly endothermic carbothermic reduction of $Al_2O_3$.

$$Al_2O_3 + 3\,CH_4 + N_2 \longrightarrow 2\,AlN + 6\,H_2 + 3\,CO$$

followed by the mildly exothermic hydrolysis of $AlN$

$$2\,AlN + 3\,H_2O \longrightarrow Al_2O_3 + 2\,NH_3$$

The by-product of these two reactions is a mixture of carbon monoxide and hydrogen which is generally referred to as synthesis gas.

### Production of Liquid Transportation Fuel from Synthesis Gas

A variety of liquid fuel products can be made from synthesis gas. Here we'll consider the production dimethyl ether $CH_3OCH_3$ (DME) which can be used an ultraclean alternative to diesel or as a blending component for gasoline. DME can be produced in a single stage over a bi-functional catalyst via the reactions

$$CO_2 + 3\,H_2 \longrightarrow CH_3OH + H_2O$$
$$CO + H_2O \longrightarrow CO_2 + H_2$$
$$2\,CH_3OH \longrightarrow CH_3OCH_3 + H_2O$$

### Process Objectives

Considering just these five reactions, is it possible to produce ammonia, and a mixture of DME and methanol without any by-product carbon dioxide? What is the net process stoichiometry?


## Solution by Linear Programming

| Species  | MW |$R_1$|$R_2$|$R_3$|$R_4$|$R_5$| Net      |
| :-----: | :-: | :-: | :-: | :-: | :-: | :-: | :-:      |
| $Al_2O_3$   | | -1  |  1  |     |     |     | 0        |
| $AlN$       | |  2  | -2  |     |     |     | 0        |
| $CH_4$      | | -3  |     |     |     |     | $\leq$ 0 |
| $N_2$       | | -1  |     |     |     |     |          |
| $H_2$       | |  6  |     | -3  |  1  |     | $\geq$ 0 |
| $CO$        | |  3  |     |     | -1  |     | 0        |
| $H_2O$      | |     | -3  |  1  | -1  |  1  |          |
| $NH_3$      | |     |  2  |     |     |     | $\geq$ 0 |
| $CO_2$      | |     |     | -1  |  1  |     | 0        |
| $CH_3OH$    | |     |     |  1  |     | -2  | $\geq$ 0 |
| $CH_3OCH_3$ | |     |     |     |     |  1  | 1        |   

### Create a dictionary for the stoichiometric coefficients in each reaction.

In [205]:
nu = dict()

nu['R1'] = {'Al2O3':-1, 'CH4':-3, 'N2':-1, 'AlN':2, 'H2':6, 'CO':3}
nu['R2'] = {'AlN':-2, 'H2O':-3, 'Al2O3':1, 'NH3':2}
nu['R3'] = {'CO2':-1, 'H2':-3, 'CH3OH':1, 'H2O':1}
nu['R4'] = {'CO':-1, 'H2O':-1, 'CO2':1, 'H2':1}
nu['R5'] = {'CH3OH':-2, 'CH3OCH3':1, 'H2O':1}
nu

{'R1': {'Al2O3': -1, 'AlN': 2, 'CH4': -3, 'CO': 3, 'H2': 6, 'N2': -1},
 'R2': {'Al2O3': 1, 'AlN': -2, 'H2O': -3, 'NH3': 2},
 'R3': {'CH3OH': 1, 'CO2': -1, 'H2': -3, 'H2O': 1},
 'R4': {'CO': -1, 'CO2': 1, 'H2': 1, 'H2O': -1},
 'R5': {'CH3OCH3': 1, 'CH3OH': -2, 'H2O': 1}}

From the stoichiometric dictionary we can create a set of reactions and species.

In [206]:
rxns = set(nu.keys())
species = set([s for r in rxns for s in nu[r].keys()])

In [207]:
def printRxn(nu):
    c2s = lambda n: '' if n==1 else str(round(n,3))+' '
    lhs = ' + '.join(["{0:s}{1:s}".format(c2s(-nu[s]),s) for s in nu.keys() if nu[s] < 0])
    rhs = ' + '.join(["{0:s}{1:s}".format(c2s(nu[s]),s) for s in nu.keys() if nu[s] > 0])
    print(' --> '.join([lhs,rhs]))
    
for r in sorted(rxns):
    print(r,end=': ')
    printRxn(nu[r])

R1: N2 + 3 CH4 + Al2O3 --> 3 CO + 6 H2 + 2 AlN
R2: 3 H2O + 2 AlN --> Al2O3 + 2 NH3
R3: CO2 + 3 H2 --> H2O + CH3OH
R4: H2O + CO --> CO2 + H2
R5: 2 CH3OH --> H2O + CH3OCH3


In [227]:
print('{0:8s}'.format(''),end='')
print(''.join(['{0:>5s}'.format(r) for r in sorted(rxns)]))
for s in sorted(species):
    print('{0:7s} '.format(s),end='')
    for r in sorted(rxns):
        ps = '{0:5d}'.format(nu[r][s]) if s in nu[r].keys() else '     '
        print(ps,end='')
    print()

           R1   R2   R3   R4   R5
Al2O3      -1    1               
AlN         2   -2               
CH3OCH3                         1
CH3OH                 1        -2
CH4        -3                    
CO          3             -1     
CO2                  -1    1     
H2          6        -3    1     
H2O             -3    1   -1    1
N2         -1                    
NH3              2               


In [234]:
!pip install cvxpy

In [316]:
import cvxpy as cvx

# extents of reaction
X = {r:cvx.Variable() for r in rxns}

# net stoichiometric coefficients
V = dict()
for s in species:
    V[s] = sum([nu[r][s]*X[r] for r in rxns if s in nu[r].keys()])

In [323]:
obj = cvx.Minimize(V['CO2'])

constraints = [
    V['Al2O3'] == 0,
    V['AlN'] == 0,
    V['CH4'] <= 0,
    V['H2'] >= 0,
    V['CO'] == 0,
    V['CH3OH'] == 0,
    V['CH3OCH3'] == 3,
    V['NH3'] >= 0]

In [324]:
prob = cvx.Problem(obj,constraints)
prob.solve()
prob.status

'optimal'

In [325]:
{r:round(X[r].value,4) for r in rxns}

{'R1': 2.0, 'R2': 2.0, 'R3': 6.0, 'R4': 6.0, 'R5': 3.0}

In [326]:
{s:round(V[s].value,4) for s in species}

{'Al2O3': -0.0,
 'AlN': 0.0,
 'CH3OCH3': 3.0,
 'CH3OH': -0.0,
 'CH4': -6.0,
 'CO': 0.0,
 'CO2': -0.0,
 'H2': -0.0,
 'H2O': -3.0,
 'N2': -2.0,
 'NH3': 4.0}

In [327]:
[c.value for c in constraints]

[True, True, True, True, True, True, True, True]